In [1]:
import os
import glob
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Align import PairwiseAligner
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

In [9]:
diffdock_df = pd.read_csv("/Users/aoxu/projects/DrugDiscovery/PoseBench/forks/DiffDock/inference/diffdock_posebusters_benchmark_output_orig_structure_2/bust_results_aggregated.csv")
df_pb = pd.read_csv("/Users/aoxu/projects/DrugDiscovery/PoseBench/forks/ICM/inference/icm_manual_posebusters_benchmark_outputs_1/bust_results_aggregated.csv")
print(diffdock_df.shape, df_pb.shape)

# If each df has columns ["protein","ligand","rmsd", "Score", "is_success"], etc.
merged_df = pd.merge(df_pb, diffdock_df,
                     on=["protein"],
                     suffixes=("_icm","_diffdock"))
print(merged_df.shape)

(215, 127) (279, 127)
(214, 253)


In [11]:
failed_proteins = merged_df[ (~merged_df["rmsd_≤_2å_icm"])]['protein'].unique()

In [ ]:
def read_fasta_file(fasta_path):
    with open(fasta_path, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            print(f">{record.id}")
            print(record.seq)

# Example usage:
seq_file_path = "/Users/aoxu/projects/DrugDiscovery/PoseBench/forks/ICM/inference/icm_manual_posebusters_benchmark_outputs_1/seqs"
read_fasta_file(seq_file_path)

## Sequence-based clustering

In [ ]:
"""
Sequence-based clustering example:
1) Read multiple protein FASTA files (one sequence per file).
2) Compute pairwise sequence identity.
3) Build a distance matrix and perform hierarchical clustering.
"""

def read_fasta_file(fasta_path):
    """
    Read a single FASTA file that contains multiple sequences.
    Returns:
      names:   list of sequence IDs (from FASTA headers)
      seqs:    list of sequence strings
    """
    names = []
    seqs = []
    with open(fasta_path, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            names.append(record.id)       # e.g. "7ZZW_KKW_chain_0"
            seqs.append(str(record.seq))  # the actual sequence
    return names, seqs

def read_fasta_files(fasta_dir):
    """
    Returns:
      names: list of protein names (filename-based)
      sequences: list of Biopython Seq objects (or strings)
    """
    names = []
    sequences = []
    for fasta_path in glob.glob(os.path.join(fasta_dir, "*.fasta")):
        protein_name = os.path.splitext(os.path.basename(fasta_path))[0]
        record = next(SeqIO.parse(fasta_path, "fasta"))  # assume 1 seq per file
        names.append(protein_name)
        sequences.append(str(record.seq))
    return names, sequences

def compute_identity(seqA, seqB):
    """
    Returns fraction identity between seqA and seqB.
    Using global pairwise alignment via Biopython PairwiseAligner.
    """
    aligner = PairwiseAligner()
    aligner.mode = "global"
    alignments = aligner.align(seqA, seqB)
    best_aln = alignments[0]
    # best_aln.score is alignment score, but we want identity
    # For simplicity, let's count matches:
    matches = 0
    aligned_length = 0
    seqA_aln = best_aln.aligned[0]  # list of (start, end) blocks in seqA
    seqB_aln = best_aln.aligned[1]  # list of (start, end) blocks in seqB
    # We'll reconstruct the alignment as best as we can:
    alignment_strA = []
    alignment_strB = []
    for (startA, endA), (startB, endB) in zip(seqA_aln, seqB_aln):
        alignment_strA.append(seqA[startA:endA])
        alignment_strB.append(seqB[startB:endB])
    aligned_seqA = "".join(alignment_strA)
    aligned_seqB = "".join(alignment_strB)

    for a, b in zip(aligned_seqA, aligned_seqB):
        if a == b:
            matches += 1
        aligned_length += 1

    return matches / aligned_length if aligned_length > 0 else 0.0

def sequence_distance_matrix(sequences):
    """
    Build NxN matrix of (1 - identity).
    """
    n = len(sequences)
    dist_matrix = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            identity = compute_identity(sequences[i], sequences[j])
            dist = 1.0 - identity
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
    return dist_matrix

def cluster_sequences(fasta_dir, selected_proteins=None):
    names, seqs = read_fasta_file(fasta_dir)
    if selected_proteins is not None:
        # Filter sequences by protein names
        selected_indices = [i for i, name in enumerate(names) if name.split('_chain')[0] in selected_proteins]
        names = [names[i] for i in selected_indices]
        seqs = [seqs[i] for i in selected_indices]
    dist_mat = sequence_distance_matrix(seqs)
    # Convert to condensed form for scipy linkage:
    # upper triangle of dist_mat (without diagonal)
    from scipy.spatial.distance import squareform
    dist_condensed = squareform(dist_mat, checks=False)

    Z = linkage(dist_condensed, method="average")  # hierarchical clustering
    # Plot dendrogram
    plt.figure(figsize=(8, 4))
    dendrogram(Z, labels=names, leaf_rotation=90)
    plt.title("Sequence-based clustering (average linkage)")
    plt.show()
    return Z

# Example usage:
seq_file_path = "/Users/aoxu/projects/DrugDiscovery/PoseBench/data/posebusters_benchmark_set/posebusters_benchmark_sequences.fasta"
cluster_sequences(seq_file_path, selected_proteins=failed_proteins)

## 2. Structure-Based Clustering (Global 3D)
Cluster proteins by structural similarity, using RMSD or TM-score from pairwise structure alignment. (TM-align)

In [ ]:
"""
Structure-based clustering:
1) For each pair of PDB files, run TM-align (or an equivalent) to get RMSD or TM-score.
2) Build an NxN distance matrix.
3) Perform hierarchical clustering or another method.
"""

import subprocess
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

def run_tmalign(pdbA, pdbB, tmalign_bin="TMalign"):
    """
    Run TM-align and parse RMSD or TM-score from stdout.
    Returns (rmsd, tm_score).
    """
    cmd = [tmalign_bin, pdbA, pdbB]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    lines = result.stdout.splitlines()

    rmsd_val = None
    tm_val = None
    for line in lines:
        if line.startswith("RMSD of the common residues"):
            # e.g. "RMSD of the common residues=   2.12"
            parts = line.split("=")
            rmsd_val = float(parts[1])
        elif line.startswith("TM-score="):
            # e.g. "TM-score= 0.45"
            parts = line.split("=")
            tm_val = float(parts[1].split()[0])
    return rmsd_val, tm_val

def load_pdb(pdb_dir, selected_proteins=None, cropped=''):
    """
    retrieve the path for the pdb files in the pdb directory
    """
    protein_names = os.listdir(pdb_dir)
    if selected_proteins is not None:
        protein_paths = [os.path.join(pdb_dir, protein_name, f"{protein_name}_protein_{cropped}.pdb") for protein_name in selected_proteins]
    else:
        protein_paths = [os.path.join(pdb_dir, protein_name, f"{protein_name}_protein_{cropped}.pdb") for protein_name in protein_names]

    return protein_paths

def build_structure_distance_matrix(pdb_dir, selected_proteins=None, cropped=''):
    """
    For each PDB in pdb_dir, run pairwise TMalign, store RMSD as distance.
    """
    names = os.listdir(pdb_dir)
    if selected_proteins is not None:
        pdb_paths = [os.path.join(pdb_dir, protein_name, f"{protein_name}_protein_{cropped}.pdb") for protein_name in selected_proteins]
    else:
        pdb_paths = [os.path.join(pdb_dir, protein_name, f"{protein_name}_protein_{cropped}.pdb") for protein_name in names]
    # names = [os.path.splitext(os.path.basename(p))[0] for p in pdb_paths]
    n = len(pdb_paths)

    dist_mat = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            print(f"Comparing {pdb_paths[i]} vs {pdb_paths[j]}")
            rmsd, tm = run_tmalign(pdb_paths[i], pdb_paths[j])
            dist = rmsd  # or (1 - tm), if you prefer to cluster by TM-score
            dist_mat[i, j] = dist
            dist_mat[j, i] = dist
    return names, dist_mat

def cluster_structures(pdb_dir, selected_proteins=None, cropped=''):
    names, dist_mat = build_structure_distance_matrix(pdb_dir)

    dist_condensed = squareform(dist_mat, checks=False)
    Z = linkage(dist_condensed, method="average")
    plt.figure(figsize=(8, 4))
    dendrogram(Z, labels=names, leaf_rotation=90)
    plt.title("Structure-based clustering (RMSD average linkage)")
    plt.show()

# Example usage:
pdb_dir = "/Users/aoxu/projects/DrugDiscovery/PoseBench/data/posebusters_benchmark_set/"
cluster_structures(pdb_dir, selected_proteins=[failed_proteins[0]], cropped='')

## 3. Pocket-Based Clustering (Feature Vectors)
Cluster proteins by pocket shape/chemistry rather than the whole structure or sequence

Requirements
1. A method to extract or compute descriptors for each pocket, e.g. fpocket, SiteMap (Schrödinger), or your own script.
2. The pocket descriptor is typically a numeric vector per pocket.

In [ ]:
"""
Pocket-based clustering:
1) For each protein's PDB, extract the binding site residues or region.
2) Compute a vector of descriptors (volume, area, polarity, etc.).
3) Use k-means or hierarchical clustering to group pockets.
"""

import os
import glob
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from itertools import cycle

def compute_pocket_features(pdb_path):
    """
    Stub function: you would call a pocket detection tool or your own algorithm.
    Return a 1D numpy array of features, e.g. [volume, polar_area, net_charge, ...]
    """
    # TODO: implement real logic
    # For demonstration, let's pretend we have random data
    import random
    feature_vector = np.array([
        random.uniform(100, 800),  # pocket volume
        random.uniform(0, 300),    # polar surface area
        random.uniform(-5, 5),     # net charge
        random.uniform(0, 1)       # hydrophobicity index, etc.
    ])
    return feature_vector

def cluster_pockets(pdb_dir, n_clusters=4):
    """
    1) Gather pocket features from each PDB.
    2) Run k-means clustering.
    """
    pdb_paths = sorted(glob.glob(os.path.join(pdb_dir, "*.pdb")))
    names = [os.path.splitext(os.path.basename(p))[0] for p in pdb_paths]

    # Compute pocket descriptor for each protein
    X = []  # array of feature vectors
    for p in pdb_paths:
        fv = compute_pocket_features(p)
        X.append(fv)
    X = np.vstack(X)  # shape: (num_proteins, num_features)

    # K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    labels = kmeans.labels_

    # Print cluster membership
    for name, lbl in zip(names, labels):
        print(f"{name} => Cluster {lbl}")

    # Optionally do a 2D plot (if we can do PCA or so)
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    Xp = pca.fit_transform(X)

    plt.figure()
    colors = cycle(["r", "g", "b", "c", "m", "y", "k"])
    for i, (name, lbl) in enumerate(zip(names, labels)):
        c = next(colors)
        plt.scatter(Xp[i, 0], Xp[i, 1], color=c, label=f"Cluster {lbl}" if i == 0 else "")
        plt.text(Xp[i, 0]+1, Xp[i, 1], name, fontsize=9)
    plt.title("Pocket-based clustering (K-means with PCA visualization)")
    plt.show()

# Example usage:
# cluster_pockets("path/to/pdb_dir", n_clusters=4)